In [11]:
import tensorflow as tf # 导入TensorFlow库 
import os # 导入OS库 

def echo_tensor(sess, tensor, prefix = ''):
    # 注意：print() 会把元素之间的逗号去掉
    if( isinstance(tensor, tf.Tensor) ):
        print( '{0} tensor.shape = {1}, tensor = {2}\r\n'.format(prefix, sess.run(tf.shape(tensor)), sess.run(tensor)) )
        # print( '{0} tensor = {1}\r\n'.format(prefix, sess.run(tensor)) )
    else:
        print( '{0} not_tensor = {1}\r\n'.format(prefix, tensor) )


# 从csv文件读取数据，加载解析，创建批次读取张量多行数据
# 调用举例: read_csv(4, ['1.csv', '2.csv'], [[0], [0.], ['']])
# 
# 如果 csv 文件中数据为：
# 1,2,3,4
# 11,12,13,14
# 21,22,23,24
# 31,32,33,34
# 41,42,43,44
# 51,52,53,54
# 
# 那么输出会是：
# [[21, 51, 1, 61], [22, 52, 2, 62], [23, 53, 3, 63], [24, 54, 4, 64]]
# 
def read_csv(batch_size, file_name, record_defaults): #从csv文件读取数据，加载解析，创建批次读取张量多行数据
    file_path = list( map(lambda name: os.path.join(os.getcwd(), name), file_name) )
    file_queue = tf.train.string_input_producer(file_path) 
    
    reader = tf.TextLineReader(skip_header_lines = 1) # reader = tf.TFRecordReader()
    key, value = reader.read(file_queue) 
    
    decoded = tf.decode_csv(value, record_defaults = record_defaults) #字符串(文本行)转换到指定默认值张量列元组，为每列设置数据类型

    return tf.train.shuffle_batch(decoded, batch_size=batch_size, capacity=batch_size * 50, min_after_dequeue=batch_size) # 读取文件，加载张量batch_size行 


with tf.Session() as sess: # 会话对象启动数据流图，搭建流程 
    print( 'Session: start' )
    tf.initialize_all_variables().run() 

    batch_data = read_csv(4, ['iris2.csv'], [[0], [0], [0], [0], ['']])
        
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess = sess, coord = coord)
    echo_tensor( sess, threads, 'threads_1' )
    
    
    try:
        # while not coord.should_stop():
            for i in range(3):
                echo_tensor( sess, batch_data, 'batch_data_' + str(i) )

            f1, f2, f3, f4, lable = batch_data
            echo_tensor(sess, f2, 'f2')
            echo_tensor(sess, lable, 'lable')
    
            
            is_class_1 = tf.to_int32(tf.equal(lable, [b'Iris-setosa']))
            is_class_2 = tf.to_int32(tf.equal(lable, [b'Iris-versicolor']))
            is_class_3 = tf.to_int32(tf.equal(lable, [b'Iris-virginica']))
    
            features = tf.pack([f1, f2, f3, f4, is_class_1])
            echo_tensor(sess, features, 'features')
            

    except tf.errors.OutOfRangeError:
        print( 'Done training -- epoch limit reached' )

    finally:
        # When done, ask the threads to stop.
        echo_tensor( sess, threads, 'threads_2' )
        coord.request_stop()        
        coord.join(threads)

    sess.close()    
    print( 'Session: close' )

Session: start
threads_1 not_tensor = [<Thread(Thread-225, started daemon 140429785720576)>, <Thread(Thread-226, started daemon 140429836076800)>, <Thread(Thread-227, started daemon 140429794113280)>, <Thread(Thread-228, started daemon 140429861254912)>, <Thread(Thread-229, started daemon 140431714141952)>, <Thread(Thread-230, started daemon 140431705749248)>, <Thread(Thread-231, started daemon 140431697356544)>, <Thread(Thread-232, started daemon 140431688963840)>, <Thread(Thread-233, started daemon 140431035709184)>, <Thread(Thread-234, started daemon 140431027316480)>, <Thread(Thread-235, started daemon 140431018923776)>, <Thread(Thread-236, started daemon 140430599517952)>, <Thread(Thread-237, started daemon 140430591125248)>, <Thread(Thread-238, started daemon 140430582732544)>, <Thread(Thread-239, started daemon 140430574339840)>, <Thread(Thread-240, started daemon 140430565947136)>, <Thread(Thread-241, started daemon 140430557554432)>, <Thread(Thread-242, started daemon 14043054